<a href="https://colab.research.google.com/github/OjhaS17/EV-Market-Segmentation-Analysis/blob/main/EV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EV Market Segmentation Analysis

In [1]:
# 1. SETUP AND LIBRARY IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Scikit-learn imports
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# SciPy for hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy import stats

# Display options
pd.set_option("display.precision", 3)
np.set_printoptions(precision=5, suppress=True)
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# 2. DATA LOADING AND PREPROCESSING

# Load data
df = pd.read_csv('/content/data.csv')
print(f"Dataset loaded: {df.shape[0]} vehicles, {df.shape[1]} features")

# Clean column names and remove index if present
if 'Unnamed: 0' in df.columns:
    df.drop('Unnamed: 0', axis=1, inplace=True)
df.columns = df.columns.str.strip()
df['Brand'] = df['Brand'].str.strip()

# Store metadata for later use
metadata = df[['Brand', 'Model']].copy()

# Separate features for analysis
numerical_features = ['AccelSec', 'TopSpeed_KmH', 'Range_Km', 'Efficiency_WhKm',
                     'FastCharge_KmH', 'Seats', 'PriceEuro']
categorical_features = ['RapidCharge', 'PowerTrain', 'PlugType', 'BodyStyle', 'Segment']

print(f"Numerical features: {len(numerical_features)}")
print(f"Categorical features: {len(categorical_features)}")

# Convert Yes/No to 1/0
df['RapidCharge'] = df['RapidCharge'].map({'Yes': 1, 'No': 0})

# One-hot encoding for categorical variables
df_encoded = pd.get_dummies(df.drop(columns=['Brand', 'Model']),
                           columns=['PowerTrain', 'PlugType', 'BodyStyle', 'Segment'],
                           drop_first=True)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded)
feature_names = df_encoded.columns.tolist()

print(f"Features after preprocessing: {X_scaled.shape[1]}")

Dataset loaded: 103 vehicles, 15 features
Numerical features: 7
Categorical features: 5
Features after preprocessing: 28


In [3]:
# 3. PRINCIPAL COMPONENT ANALYSIS (PCA)

# Perform PCA
pca = PCA()
X_pca_full = pca.fit_transform(X_scaled)

# Calculate variance explained
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Find optimal number of components (95% variance)
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
n_components_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_components_85 = np.argmax(cumulative_variance >= 0.85) + 1

print(f"Variance explained:")
print(f"   - 85% variance: {n_components_85} components")
print(f"   - 90% variance: {n_components_90} components")
print(f"   - 95% variance: {n_components_95} components")

# Interactive variance plot
fig_pca = make_subplots(rows=1, cols=2,
                       subplot_titles=('Individual Variance Explained',
                                     'Cumulative Variance Explained'))

# Individual variance
fig_pca.add_trace(
    go.Bar(x=list(range(1, 16)), y=explained_variance[:15],
           name='Individual', marker_color='lightblue'),
    row=1, col=1
)

# Cumulative variance
fig_pca.add_trace(
    go.Scatter(x=list(range(1, len(cumulative_variance)+1)),
               y=cumulative_variance, mode='lines+markers',
               name='Cumulative', line=dict(color='darkblue')),
    row=1, col=2
)

# Add reference lines
fig_pca.add_hline(y=0.85, line_dash="dash", line_color="green",
                  annotation_text="85%", row=1, col=2)
fig_pca.add_hline(y=0.90, line_dash="dash", line_color="orange",
                  annotation_text="90%", row=1, col=2)
fig_pca.add_hline(y=0.95, line_dash="dash", line_color="red",
                  annotation_text="95%", row=1, col=2)

fig_pca.update_layout(height=400, title_text="PCA Variance Analysis", showlegend=False)
fig_pca.show()

# Select components for 90% variance (balance between dimensionality reduction and information retention)
n_components = n_components_90
X_pca = X_pca_full[:, :n_components]
print(f"Selected {n_components} components explaining {cumulative_variance[n_components-1]:.1%} of variance")

Variance explained:
   - 85% variance: 12 components
   - 90% variance: 14 components
   - 95% variance: 16 components


Selected 14 components explaining 91.7% of variance


In [4]:
# 4. OPTIMAL CLUSTER NUMBER DETERMINATION

# Test range of clusters
k_range = range(2, 11)
metrics = {
    'k': [],
    'inertia': [],
    'silhouette': [],
    'calinski': [],
    'davies_bouldin': []
}

for k in k_range:
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10, random_state=42)
    labels = kmeans.fit_predict(X_pca)

    metrics['k'].append(k)
    metrics['inertia'].append(kmeans.inertia_)
    metrics['silhouette'].append(silhouette_score(X_pca, labels))
    metrics['calinski'].append(calinski_harabasz_score(X_pca, labels))
    metrics['davies_bouldin'].append(davies_bouldin_score(X_pca, labels))

# Create evaluation plots
fig_metrics = make_subplots(rows=2, cols=2,
                           subplot_titles=('Elbow Method', 'Silhouette Score',
                                         'Calinski-Harabasz Score', 'Davies-Bouldin Score'))

# Elbow plot
fig_metrics.add_trace(go.Scatter(x=metrics['k'], y=metrics['inertia'],
                                mode='lines+markers', name='Inertia'),
                     row=1, col=1)

# Silhouette score
fig_metrics.add_trace(go.Scatter(x=metrics['k'], y=metrics['silhouette'],
                                mode='lines+markers', name='Silhouette',
                                line=dict(color='green')),
                     row=1, col=2)

# Calinski-Harabasz score
fig_metrics.add_trace(go.Scatter(x=metrics['k'], y=metrics['calinski'],
                                mode='lines+markers', name='Calinski',
                                line=dict(color='orange')),
                     row=2, col=1)

# Davies-Bouldin score (lower is better)
fig_metrics.add_trace(go.Scatter(x=metrics['k'], y=metrics['davies_bouldin'],
                                mode='lines+markers', name='Davies-Bouldin',
                                line=dict(color='red')),
                     row=2, col=2)

fig_metrics.update_layout(height=700, title_text="Clustering Metrics for K Selection",
                         showlegend=False)
fig_metrics.show()

# Select optimal k based on multiple metrics
# Normalize scores for comparison
silhouette_norm = (metrics['silhouette'] - np.min(metrics['silhouette'])) / (np.max(metrics['silhouette']) - np.min(metrics['silhouette']))
calinski_norm = (metrics['calinski'] - np.min(metrics['calinski'])) / (np.max(metrics['calinski']) - np.min(metrics['calinski']))
davies_norm = 1 - (metrics['davies_bouldin'] - np.min(metrics['davies_bouldin'])) / (np.max(metrics['davies_bouldin']) - np.min(metrics['davies_bouldin']))

# Combined score
combined_scores = [(s + c + d) / 3 for s, c, d in zip(silhouette_norm, calinski_norm, davies_norm)]
optimal_k = metrics['k'][np.argmax(combined_scores)]

print(f"Optimal number of clusters: {optimal_k}")

Optimal number of clusters: 10


In [5]:
# 5. COMPARATIVE CLUSTERING ANALYSIS

# Initialize models
models = {
    'K-Means': KMeans(n_clusters=optimal_k, init='k-means++', n_init=10, random_state=42),
    'Hierarchical': AgglomerativeClustering(n_clusters=optimal_k, linkage='ward'),
    'GMM': GaussianMixture(n_components=optimal_k, random_state=42)
}

# Fit models and evaluate
results = []
cluster_labels = {}

for name, model in models.items():
    if name == 'GMM':
        model.fit(X_pca)
        labels = model.predict(X_pca)
    else:
        labels = model.fit_predict(X_pca)

    cluster_labels[name] = labels

    # Calculate metrics
    sil_score = silhouette_score(X_pca, labels)
    cal_score = calinski_harabasz_score(X_pca, labels)
    db_score = davies_bouldin_score(X_pca, labels)

    results.append({
        'Algorithm': name,
        'Silhouette': sil_score,
        'Calinski-Harabasz': cal_score,
        'Davies-Bouldin': db_score
    })

    print(f"\n{name}:")
    print(f"  Silhouette Score: {sil_score:.4f}")
    print(f"  Calinski-Harabasz: {cal_score:.4f}")
    print(f"  Davies-Bouldin: {db_score:.4f}")

# Create comparison dataframe
results_df = pd.DataFrame(results)

# Normalize scores for comparison
results_df['Silhouette_norm'] = (results_df['Silhouette'] - results_df['Silhouette'].min()) / (results_df['Silhouette'].max() - results_df['Silhouette'].min())
results_df['Calinski_norm'] = (results_df['Calinski-Harabasz'] - results_df['Calinski-Harabasz'].min()) / (results_df['Calinski-Harabasz'].max() - results_df['Calinski-Harabasz'].min())
results_df['Davies_norm'] = 1 - (results_df['Davies-Bouldin'] - results_df['Davies-Bouldin'].min()) / (results_df['Davies-Bouldin'].max() - results_df['Davies-Bouldin'].min())
results_df['Overall_Score'] = (results_df['Silhouette_norm'] + results_df['Calinski_norm'] + results_df['Davies_norm']) / 3

# Select best model
best_model = results_df.loc[results_df['Overall_Score'].idxmax(), 'Algorithm']
best_labels = cluster_labels[best_model]

print(f"\nBest performing model: {best_model}")
print("Model Comparison Summary:")
print(results_df[['Algorithm', 'Overall_Score']].sort_values('Overall_Score', ascending=False))


K-Means:
  Silhouette Score: 0.3882
  Calinski-Harabasz: 23.7736
  Davies-Bouldin: 0.8705

Hierarchical:
  Silhouette Score: 0.3824
  Calinski-Harabasz: 22.9728
  Davies-Bouldin: 0.9282

GMM:
  Silhouette Score: 0.3882
  Calinski-Harabasz: 23.7736
  Davies-Bouldin: 0.8705

Best performing model: K-Means
Model Comparison Summary:
      Algorithm  Overall_Score
0       K-Means         1.0000
2           GMM         1.0000
1  Hierarchical         0.0000


In [6]:
# 6. CLUSTERING VISUALIZATION

# Add cluster labels and PCA components to dataframe
df['Cluster'] = best_labels
df['PC1'] = X_pca_full[:, 0]
df['PC2'] = X_pca_full[:, 1]
df['PC3'] = X_pca_full[:, 2] if X_pca_full.shape[1] > 2 else 0

# Create interactive 3D visualization
fig_3d = px.scatter_3d(df, x='PC1', y='PC2', z='PC3',
                       color='Cluster',
                       hover_data=['Brand', 'Model', 'PriceEuro', 'Range_Km'],
                       title=f'3D Cluster Visualization ({best_model})',
                       labels={'PC1': 'PC1 (Performance)',
                               'PC2': 'PC2 (Efficiency)',
                               'PC3': 'PC3 (Utility)'},
                       color_continuous_scale='Viridis')

fig_3d.update_traces(marker=dict(size=5))
fig_3d.show()

In [7]:
# 7. FEATURE IMPORTANCE ANALYSIS

# Train Random Forest on original features to predict clusters
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_scaled, best_labels)

# Get feature importances
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

# Create importance plot
fig_importance = px.bar(importance_df.head(15),
                       x='Importance', y='Feature',
                       orientation='h',
                       title='Top 15 Features Driving Market Segmentation',
                       color='Importance',
                       color_continuous_scale='Blues')

fig_importance.update_layout(yaxis={'categoryorder':'total ascending'})
fig_importance.show()

print("\nTop 10 Most Important Features:")
print(importance_df.head(10))


Top 10 Most Important Features:
            Feature  Importance
22        Segment_C      0.1718
21        Segment_B      0.0922
23        Segment_D      0.0880
7         PriceEuro      0.0859
1      TopSpeed_KmH      0.0745
3   Efficiency_WhKm      0.0672
24        Segment_E      0.0601
4    FastCharge_KmH      0.0530
2          Range_Km      0.0455
0          AccelSec      0.0432


In [8]:
# 8. DETAILED SEGMENT PROFILING

# Profile each segment
for cluster in sorted(df['Cluster'].unique()):
    cluster_data = df[df['Cluster'] == cluster]
    n_vehicles = len(cluster_data)

    print(f"\n{'='*50}")
    print(f"CLUSTER {cluster} - {n_vehicles} vehicles ({n_vehicles/len(df)*100:.1f}%)")
    print(f"{'='*50}")

    # Price analysis
    price_stats = cluster_data['PriceEuro'].describe()
    print(f"\nPrice Profile:")
    print(f"   Average: €{price_stats['mean']:,.0f}")
    print(f"   Range: €{price_stats['min']:,.0f} - €{price_stats['max']:,.0f}")

    # Performance analysis
    print(f"\nPerformance Profile:")
    print(f"   Acceleration: {cluster_data['AccelSec'].mean():.1f}s (0-100km/h)")
    print(f"   Top Speed: {cluster_data['TopSpeed_KmH'].mean():.0f} km/h")

    # Range and efficiency
    print(f"\nRange & Efficiency:")
    print(f"   Range: {cluster_data['Range_Km'].mean():.0f} km")
    print(f"   Efficiency: {cluster_data['Efficiency_WhKm'].mean():.0f} Wh/km")
    print(f"   Fast Charge: {cluster_data['FastCharge_KmH'].mean():.0f} km/h")

    # Top brands
    print(f"\nTop Brands:")
    top_brands = cluster_data['Brand'].value_counts().head(5)
    for brand, count in top_brands.items():
        print(f"   - {brand}: {count} models ({count/n_vehicles*100:.0f}%)")

    # Body styles
    print(f"\nPrimary Body Styles:")
    body_cols = [col for col in df.columns if 'BodyStyle_' in col]
    if body_cols:
        body_dist = cluster_data[body_cols].sum().sort_values(ascending=False).head(3)
        for style, count in body_dist.items():
            style_name = style.replace('BodyStyle_', '')
            print(f"   - {style_name}: {int(count)} vehicles")


CLUSTER 0 - 25 vehicles (24.3%)

Price Profile:
   Average: €33,267
   Range: €20,129 - €41,906

Performance Profile:
   Acceleration: 9.1s (0-100km/h)
   Top Speed: 148 km/h

Range & Efficiency:
   Range: 257 km
   Efficiency: 168 Wh/km
   Fast Charge: 278 km/h

Top Brands:
   - Renault: 3 models (12%)
   - Opel: 3 models (12%)
   - Kia: 3 models (12%)
   - Hyundai: 2 models (8%)
   - Honda: 2 models (8%)

Primary Body Styles:

CLUSTER 1 - 7 vehicles (6.8%)

Price Profile:
   Average: €131,618
   Range: €102,945 - €180,781

Performance Profile:
   Acceleration: 3.4s (0-100km/h)
   Top Speed: 251 km/h

Range & Efficiency:
   Range: 425 km
   Efficiency: 203 Wh/km
   Fast Charge: 779 km/h

Top Brands:
   - Porsche: 5 models (71%)
   - Lucid: 1 models (14%)
   - Audi: 1 models (14%)

Primary Body Styles:

CLUSTER 2 - 15 vehicles (14.6%)

Price Profile:
   Average: €58,488
   Range: €46,380 - €69,484

Performance Profile:
   Acceleration: 5.4s (0-100km/h)
   Top Speed: 202 km/h

Range & 

In [9]:
# 9. EXPORT RESULTS

# Save segmented data
df_export = df.copy()
df_export = df_export[['Brand', 'Model', 'Cluster'] + numerical_features + categorical_features]
df_export.to_csv('ev_segments_enhanced.csv', index=False)
print("Segmented data saved to 'ev_segments_enhanced.csv'")

# Save segment summary
segment_summary = []
for cluster in sorted(df['Cluster'].unique()):
    cluster_data = df[df['Cluster'] == cluster]
    summary = {
        'Cluster': cluster,
        'Size': len(cluster_data),
        'Percentage': f"{len(cluster_data)/len(df)*100:.1f}%",
        'Avg_Price': f"€{cluster_data['PriceEuro'].mean():,.0f}",
        'Avg_Range': f"{cluster_data['Range_Km'].mean():.0f} km",
        'Avg_Acceleration': f"{cluster_data['AccelSec'].mean():.1f}s",
        'Top_Brands': ', '.join(cluster_data['Brand'].value_counts().head(3).index)
    }
    segment_summary.append(summary)

segment_summary_df = pd.DataFrame(segment_summary)
segment_summary_df.to_csv('segment_summary.csv', index=False)
print("Segment summary saved to 'segment_summary.csv'")

# Save model comparison
results_df[['Algorithm', 'Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin', 'Overall_Score']].to_csv('model_comparison_enhanced.csv', index=False)
print("Model comparison saved to 'model_comparison_enhanced.csv'")

# Save feature importance
importance_df.to_csv('feature_importance_enhanced.csv', index=False)
print("Feature importance saved to 'feature_importance_enhanced.csv'")

Segmented data saved to 'ev_segments_enhanced.csv'
Segment summary saved to 'segment_summary.csv'
Model comparison saved to 'model_comparison_enhanced.csv'
Feature importance saved to 'feature_importance_enhanced.csv'
